### Import Modules

In [84]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import pickle
from project4 import pickle_stuff

### Scrape G2

In [85]:
g2_star_ratings = []
g2_product_reviews = []
def get_reviews_g2():
    driver = webdriver.Chrome('/Applications/chromedriver')
    page = 0
    while page <= 3:
        page += 1
        g2_url = 'https://www.g2.com/products/' + 'ncr-aloha-pos' + '/reviews?page=' + str(page)
        driver.get(g2_url)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')        
        review = str(soup.find_all('div', {'itemprop': 'reviewBody'}))
        g2_product_reviews.append(review)
        
        star = str(soup.find_all('div', {'class':'f-1 d-f ai-c mb-half--small-only'}))
        g2_star_ratings.append(star)
        
        all_g2_ratings = []
        for rating in g2_star_ratings:
            result = re.findall('(?<=stars large stars-)(.*?)(?=")', rating)
            all_g2_ratings.append(result)
        
    time.sleep(5)
    driver.close()
    
get_reviews_g2()

#### Grab Star Ratings from HTML

In [86]:
all_g2_ratings = []
for rating in g2_star_ratings:
    result = re.findall('(?<=stars large stars-)(.*?)(?=")', rating)
    all_g2_ratings.append(result)
stars = [item for sublist in all_g2_ratings for item in sublist]

ratings = []
for star in stars:
    ratings.append(int(star))  

#### Grab Reviews from HTML

In [87]:
helper = []
for string in g2_product_reviews:
    sub_string = string.split('div>, ')
    helper.append(sub_string)
reviews = [item for sublist in helper for item in sublist]
len(reviews)

165

In [58]:
#g2_product_reviews

#### Combine Reviewer Id and Star Rating in DataFrame

In [88]:
reviewer = list(range(165))
ratings_table = pd.DataFrame(zip(reviewer, ratings)).rename(columns={0: "reviewer", 1: "rating"})

#### Clean Up Reviews

In [89]:
all_g2_reviews = []
for review in reviews:
    result = re.findall('(?<=text">).*?(?=<)', review)
    all_g2_reviews.append(result)

In [3]:
len(all_g2_reviews)

#### Merge Reviews to DataFrame

In [91]:
g2_reviews_flat = [item for sublist in all_g2_reviews for item in sublist]

In [92]:
g2_reviews_flat_mod = {item:[index] for index,sublist in enumerate(all_g2_reviews) for item in sublist}

In [93]:
x = pd.DataFrame(g2_reviews_flat_mod).T.reset_index().rename(columns={"index": "text", 0: "reviewer"})

In [94]:
df = x.merge(ratings_table, how='left', on = 'reviewer')

In [49]:
# filepath = '/Users/tim/src/Metis/Project_4/data/interim/aloha_g2_reviews.pkl'
# with open(filepath, 'wb') as pkl:
#     pickle.dump(df, pkl)

### Scrape Capterra

In [95]:
capt_product_reviews = []
def get_reviews_capterra():
    driver = webdriver.Chrome('/Applications/chromedriver')
   
    capt_url = 'https://www.capterra.com/p/178748/Aloha/#reviews'
    driver.get(capt_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')        
    review = str(soup.find_all('div', {'class': "review-comments color-text"}))
    capt_product_reviews.append(review)

    time.sleep(5)
    driver.close()
    
get_reviews_capterra()

#### Grab Reviews from HTML

In [101]:
helper = []
for string in capt_product_reviews:
    sub_string = string.split('div>, ')
    helper.append(sub_string)
reviews_capt = [item for sublist in helper for item in sublist]

In [102]:
all_capt_reviews = []
for review in reviews_capt:
    result = re.findall('(?<=\/strong>).*?(?=<)', review)
    all_capt_reviews.append(result)

In [104]:
len(all_capt_reviews)

29

#### Combine Reviewer Id and Review in DataFrame

In [105]:
capt_reviews_flat = [item for sublist in all_capt_reviews for item in sublist]
capt_reviews_flat_mod = {item:[index+166] for index,sublist in enumerate(all_capt_reviews) for item in sublist}
x_capt = pd.DataFrame(capt_reviews_flat_mod).T.reset_index().rename(columns={"index": "text", 0: "reviewer"})

In [107]:
df= df.append(x_capt)

In [108]:
filepath = '/Users/tim/src/Metis/Project_4/data/interim/aloha_reviews2.pkl'
with open(filepath, 'wb') as pkl:
    pickle.dump(df, pkl)